In [2]:
import tabula as tb
import pandas as pd
import numpy as np
import re

## Race

In [3]:
df = tb.read_pdf("../Data/Race/2022-VAL-RAC.pdf", area = (120, 0, 500, 222), columns=[72, 78, 90, 110], pages = "1")[0]
df

Got stderr: Feb 22, 2023 3:47:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 22, 2023 3:47:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 22, 2023 3:47:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 22, 2023 3:47:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Pos,Unnamed: 0,Unnamed: 1,Unnamed: 2,Rider N
0,1,NaN,25,42,Alex RINS
1,2,NaN,20,33,Brad BINDER
2,3,NaN,16,89,Jorge MARTIN
3,4,NaN,13,20,Fabio QUARTARARO
4,5,NaN,11,88,Miguel OLIVEIRA
5,6,NaN,10,36,Joan MIR
6,7,NaN,9,10,Luca MARINI
7,8,NaN,8,23,Enea BASTIANINI
8,9,NaN,7,63,Francesco BAGNAIA
9,10,NaN,6,21,Franco MORBIDELLI


In [4]:
# remove all instances of 'Pos' that are not an integer
df2 = df[pd.to_numeric(df['Pos'], errors='coerce').notnull()]
df2.tail()

,Pos,Unnamed: 0,Unnamed: 1,Unnamed: 2,Rider N
12,13,NaN,3,87,Remy GARDNER
13,14,NaN,2,30,Takaaki NAKAGAMI
14,15,NaN,1,49,Fabio DI GIANNANTONIO
15,16,NaN,NaN,35,Cal CRUTCHLOW
16,17,NaN,NaN,73,Alex MARQUEZ


In [5]:
# remove unnecessary columns (which are the 2nd and 3rd columns)
race = df2[['Pos', 'Unnamed: 2', 'Rider N']]
race.rename(columns={'Unnamed: 2': 'Number', 'Rider N': 'Name'}, inplace=True)
race.head()

/tmp/ipykernel_27618/724783410.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race.rename(columns={'Unnamed: 2': 'Number', 'Rider N': 'Name'}, inplace=True)


,Pos,Number,Name
0,1,42,Alex RINS
1,2,33,Brad BINDER
2,3,89,Jorge MARTIN
3,4,20,Fabio QUARTARARO
4,5,88,Miguel OLIVEIRA


## Free Practice

In [12]:
filename = "../Data/FP/2022-VAL-FP4.pdf"

In [13]:
# getting the data for the free practice pdf is problematic because the table is split into two sides in one page
dfl = tb.read_pdf(filename, area = (20, 0, 730, 133), columns = [79], pandas_options = {'header': False}, pages = 'all', silent=True)  # left side
dfr = tb.read_pdf(filename, area = (20, 318, 730, 399), columns = [340], pages = 'all', silent=True)  # right side

dfl[0].head(20)

,Circui,t Ricardo Tor
0,NaN,4005 m.
1,* La,p / Sector time
2,P Cro,ssing the finis
3,**Ty,re data subject
4,Lap,Lap Time
5,1st,88 MigRed
6,Run,# 1 Fro
7,1,1'56.493
8,2,1'32.274
9,3,1'31.184


In [14]:
# next step is to combine all those data together, from a list of dataframes to one huge dataframe
# the combination goes like this: on the first page, we take the data from the left side first, then to the right side, then to the next page
# therefore, [dfl[0], dfr[0], dfl[1], dfr[1], dfl[2], ...]
combine_df = []
for i in range(len(dfl)):
    combine_df.append(dfl[i])
    combine_df.append(dfr[i])
df = pd.DataFrame(np.concatenate(combine_df, axis=0), columns=dfl[0].columns)
df

,Circui,t Ricardo Tor
0,NaN,4005 m.
1,* La,p / Sector time
2,P Cro,ssing the finis
3,**Ty,re data subject
4,Lap,Lap Time
...,...,...
433,10,5'29.713
434,11,1'33.822
435,12,1'38.672
436,Lap,Lap Time


In [15]:
# next step is to remove unnecessary values and clean the data
df2 = df.copy()
df2.rename(columns={df2.columns[0]: 'Lap Number', df2.columns[1]: 'Lap Time'}, inplace=True)

# remove instances that doesn't contain any digit from 'Lap Number'
df2['Lap Number'].replace('^([^0-9]*)$', '', regex=True, inplace=True) 
df2['Lap Number'].replace('', np.nan, inplace=True)  # dunno if this two-step replacement is necessary, lazy to check

df2.dropna(inplace=True)
df2.reset_index(drop=True, inplace=True)

#[^\'.\d]
#\d{0,2}'\d\d\.\d\d\d.*
df2

,Lap Number,Lap Time
0,1st,88 MigRed
1,1,1'56.493
2,2,1'32.274
3,3,1'31.184
4,4,1'31.014
...,...,...
372,8,1'32.488
373,9,1'40.120 P
374,10,5'29.713
375,11,1'33.822


In [16]:
# get rider number for identification purposes
_temp = pd.to_numeric(df2['Lap Number'], errors='coerce')
riders = df2[_temp.isna()].copy()

riders['Lap Time'].replace('\D+', '', regex=True, inplace=True) 
riders['Lap Time'].replace('', np.nan, inplace=True)
riders.dropna(inplace=True)

riders.head()
#df2['Lap Number'].str.findall(r'\d{0,2}[a-z][a-z]')

,Lap Number,Lap Time
0,1st,88
18,2nd,20
38,3rd,89
57,4th,44
73,5th,5


In [17]:
rider_number = riders['Lap Time']  # this will be the rider's identity, make this as a column

In [18]:
# now we need to get all the lap times for each rider, this is done by iterating
# through df2['Lap Time'] and using rider_index as a way to separate which lap time belongs to which rider
rider_index = riders.index
laps = df2['Lap Time']

k = 0  # this is a pointer to the index of rider_index
"""
[0, 17, 33, 48, 68, 84, 101, 117, 134, 154, 171, 188, 206, 222, 240, 260, 279, 293, 310, 327, 346, 364, 380, 394]
for example, when i reaches 17, k points to the 1st index,
and when i reaches 33, k points to the 2nd index, etc
"""
laps_list = [[] for i in range(len(rider_index))]
for i in range(1, len(laps)):  # we skip i=0 because we know i=0 doesn't contain any laptime
    if i in rider_index:
        k += 1
    elif re.search(r"\d{0,2}'\d\d\.\d\d\d.*", laps[i]):
        laps_list[k].append(laps[i])

In [19]:
df3 = pd.DataFrame(laps_list).transpose()
df3.columns = rider_number
# obviously, the first lap never counts, so we can easily remove them
df3.drop(0, inplace=True)
df3.head()

Lap Time,88,20,89,44,5,10,43,36,33,12,...,72,93,21,35,73,42,30,49,87,40
1,1'32.274,4'04.205,1'31.737,1'32.902,1'32.623,1'32.634 *,1'32.568,1'33.279,1'32.578,1'33.115,...,1'32.781,1'34.707,1'33.022,1'32.550,1'32.961,1'33.125,1'38.619,1'32.448,1'33.796,1'34.296
2,1'31.184,1'33.165 *,1'31.572,1'31.528,1'31.454,1'32.209,1'31.339,1'32.399,1'31.651,1'32.444,...,1'31.730,1'32.050,1'31.923,1'31.828 *,1'32.482,1'32.027,1'32.884,1'32.035,1'32.828,1'32.882
3,1'31.014,1'31.520,1'34.938,1'44.080,1'31.601,1'44.012,1'31.728,1'31.943,1'31.561,1'31.836,...,1'31.389,1'31.679,1'31.820,1'31.513,1'31.717,1'31.849,1'32.345,1'32.060,1'32.621,1'32.350
4,1'30.762,1'31.125,1'31.327,1'33.773,1'31.072,1'31.252,1'31.879 *,1'36.159,1'31.848,1'31.410,...,1'32.027,1'31.523,1'31.752,1'38.233,1'31.676,1'31.732,1'32.771 *,1'31.912,1'31.917,1'32.447
5,1'30.851,1'30.996,1'31.984,1'31.934 *,1'32.003,1'31.096,1'31.695,1'31.972,1'35.953,1'36.007 P,...,1'33.200,1'31.431,1'31.740,1'31.449,1'31.828,1'40.557,1'32.098,1'31.808,1'44.140,1'41.054


In [20]:
# these are functions to convert laptime format
def lap_to_sec(lap):
    # converts laptime format: from --'---.--- string type to seconds float type
    minsec = lap.split("'")
    sec = round(int(minsec[0]) * 60 + float(minsec[1]), 3)
    return sec

def sec_to_lap(sec):
    # converts laptime format: from seconds float type to --'---.--- string type
    min = 0
    while sec >= 60:
        sec -= 60
        min += 1
    sec = format(round(sec, 3), ".3f")
    lap = str(min) + "'" + str(sec).zfill(6)
    return lap

In [21]:
# next, we remove laptimes that are the first laps out of pits, 
# the laps where the rider enters the pit (marked with 'P'),
# and the invalid laps (marked with '*').
row_len = df3.shape[0]
col_len = df3.shape[1]

pit = False
for i in range(col_len):
    for j in range(row_len):
        #.iat accesses dataframe by [row, column]
        laptime = df3.iat[j, i]

        # invalid laptimes converted to None
        if laptime == None:
            break
        if "P" in laptime:
            pit = True
            df3.iat[j, i] = None
        elif pit:
            df3.iat[j, i] = None
            pit = False
        elif "*" in laptime:
            df3.iat[j, i] = None

        # valid laptimes converted to float
        else:
            df3.iat[j, i] = lap_to_sec(laptime)
df3.head()

Lap Time,88,20,89,44,5,10,43,36,33,12,...,72,93,21,35,73,42,30,49,87,40
1,92.274,None,91.737,92.902,None,None,92.568,93.279,92.578,93.115,...,92.781,None,None,None,None,None,None,None,None,94.296
2,91.184,None,91.572,91.528,91.454,92.209,91.339,92.399,91.651,92.444,...,91.73,92.05,91.923,None,92.482,92.027,92.884,92.035,92.828,92.882
3,91.014,91.52,94.938,104.08,91.601,104.012,91.728,91.943,91.561,91.836,...,91.389,91.679,91.82,91.513,91.717,91.849,92.345,92.06,92.621,92.35
4,90.762,91.125,91.327,93.773,91.072,91.252,None,96.159,91.848,91.41,...,92.027,91.523,91.752,98.233,91.676,91.732,None,91.912,91.917,92.447
5,90.851,90.996,91.984,None,92.003,91.096,91.695,91.972,95.953,None,...,93.2,91.431,91.74,91.449,91.828,100.557,92.098,91.808,104.14,101.054


### Laptime outlier detection
Unlike races where riders push from start to finish, riders sometimes slow down in free practice, causing them to record laps much slower than what they're capable of. This is obviously not reflective of what they do in races, so we need to remove the slow laps. However, the laptimes in free practice are not normally distributed (since riders slow down, skewing the average laptime much more to the slower side), so we can't use the usual 1.5 * IQR approximation.

Naturally, riders might slow down towards the end of the session due to tyre degradation -- it is a race simulation practice after all -- and sometimes riders make slight mistakes when they're pushing. So, taking all external factors into account, I will use a 102% threshold to detect outliers. Any laptime that is above 102% of the rider's best time will be removed.

In [22]:
minimum_lap = df3.min(skipna=True)
threshold = minimum_lap * 1.02
df4 = df3[df3 <= threshold]  # remove all values above the threshold
df4.head()

Lap Time,88,20,89,44,5,10,43,36,33,12,...,72,93,21,35,73,42,30,49,87,40
1,92.274,NaN,91.737,NaN,NaN,NaN,92.568,NaN,92.578,NaN,...,92.781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91.184,NaN,91.572,91.528,91.454,92.209,91.339,92.399,91.651,92.444,...,91.73,92.05,91.923,NaN,92.482,92.027,92.884,92.035,92.828,92.882
3,91.014,91.52,NaN,NaN,91.601,NaN,91.728,91.943,91.561,91.836,...,91.389,91.679,91.82,91.513,91.717,91.849,92.345,92.06,92.621,92.35
4,90.762,91.125,91.327,NaN,91.072,91.252,NaN,NaN,91.848,91.41,...,92.027,91.523,91.752,NaN,91.676,91.732,NaN,91.912,91.917,92.447
5,90.851,90.996,91.984,NaN,92.003,91.096,91.695,91.972,NaN,NaN,...,93.2,91.431,91.74,91.449,91.828,NaN,92.098,91.808,NaN,NaN


### Final Step
Find average laptime for each rider, then sort from fastest to slowest

In [23]:
df5 = df4.mean().sort_values().reset_index()
df5.rename(columns={"Lap Time": "Number", 0: "Lap Time"}, inplace=True)
df5.head()

,Number,Lap Time
0,20,91.2900
1,88,91.3060
2,44,91.4556
3,89,91.4581
4,43,91.5996


### Merge free practice with race data
Note: there is a high chance that crash(es) occur(s) during the race, leaving the number of riders finishing the race lesser than the number of riders completing the free practice. For the sake of simplicity, we'll just remove them from the data.

In [29]:
# remove DNF and DNS riders
df6 = df5[df5["Number"].isin(race["Number"])]    # turn them to NaN
fp = df6[df6["Number"].notna()].reset_index()    # remove NaN

# how about riders who finished the race but did NOT participate in free practice?
race2 = race[race["Number"].isin(fp["Number"])]
race3 = race2[race2["Number"].notna()].reset_index()

final = pd.DataFrame({"fp": fp["Number"], "race": race3["Number"]})
final

,fp,race
0,20,42
1,88,33
2,89,89
3,10,20
4,36,88
5,33,36
6,21,10
7,63,23
8,35,63
9,42,21
